In [1]:
# 自动求导
import torch
x = torch.arange(4.0) # 创建一个一维张量，包含4个元素，数据类型为float32
x

tensor([0., 1., 2., 3.])

In [2]:
# 要计算 y 关于 x 的梯度之前，需要有个地方存储梯度信息
x.requires_grad_(True) # 设置 x 以便进行梯度计算
x.grad

In [3]:
y = 2 * torch.dot(x, x)  # dot 函数计算两个张量的点积
y

tensor(28., grad_fn=<MulBackward0>)

In [4]:
# 通过反向传播函数来自动计算 y 关于 x 的每个分量的梯度
y.backward()
x.grad

tensor([ 0.,  4.,  8., 12.])

In [5]:
x.grad == 4 * x

tensor([True, True, True, True])

In [6]:
# 默认情况下，pytorch 会累积梯度，需要清除
x.grad.zero_()
y = x.sum()
y.backward()
x.grad

tensor([1., 1., 1., 1.])

In [7]:
# 对于非标量变量，需要传入一个 gradient 参数
x.grad.zero_()
y = x * x
y.sum().backward()
x.grad

tensor([0., 2., 4., 6.])

### detach 方法
```
在设置了 requires_grad=True 后，PyTorch 会跟踪该张量（x）被执行的操作
比如 y = x * 2，时，PyTorch 会记住 “y 是 x 乘以 2 得来的”
调用 backward 时，PyTorch 会根据这些操作反向传播计算梯度

当执行 z = y.detach() 时，PyTorch 会创建一个新的张量 z
z 的底层内存和 y 完全相同，即修改 z 的值会影响 y，反之亦然
但是 z 是脱离计算图的
 1. z 的 requires_grad 属性被设置为 False
 2. z 没有任何梯度函数
 3. PyTorch 不知道 z 是怎么计算的来的
即梯度将无法通过 z 反向传播到 x 和 y
```

In [8]:
x.grad.zero_()
y = x * x
u = y.detach()
z = u * x
z.sum().backward()
x.grad == u

tensor([True, True, True, True])

In [9]:
x.grad.zero_()
y.sum().backward()
x.grad == 2 * x

tensor([True, True, True, True])

In [19]:
# 对复杂的 python 函数求导
def f(a):
    b = a * 2
    while b.norm() < 1000: # norm 函数计算张量的范数，范数是张量元素的平方和再开根号
        b = b * 2
    if b.sum() > 0:
        c = b
    else:
        c = 100 * b
    return c

a = torch.randn(size=(), requires_grad=True) # 创建一个标量张量, size=() 表示0维，即标量
a
d = f(a)
d.backward()
print(a.grad)
print(a.grad == d / a)
a.grad.zero_()

tensor(2048.)
tensor(True)


tensor(0.)